In [ ]:
!pip install magenta

     |████████████████████████████████| 1.4 MB 6.2 MB/s 
     |████████████████████████████████| 352 kB 41.0 MB/s 
     |████████████████████████████████| 1.6 MB 35.5 MB/s 
     |████████████████████████████████| 210 kB 20.2 MB/s 
     |████████████████████████████████| 2.3 MB 39.9 MB/s 
     |████████████████████████████████| 1.4 MB 51.2 MB/s 
     |████████████████████████████████| 69 kB 7.3 MB/s 
     |████████████████████████████████| 5.6 MB 44.4 MB/s 
     |████████████████████████████████| 3.6 MB 37.4 MB/s 
     |████████████████████████████████| 254 kB 42.8 MB/s 
     |████████████████████████████████| 204 kB 40.5 MB/s 
     |████████████████████████████████| 90 kB 9.1 MB/s 
     |████████████████████████████████| 20.2 MB 1.1 MB/s 
     |████████████████████████████████| 1.1 MB 41.7 MB/s 
     |████████████████████████████████| 367 kB 64.2 MB/s 
     |████████████████████████████████| 366 kB 46.6 MB/s 
     |████████████████████████████████| 981 kB 45.2 MB/s 
     |█████████████

In [ ]:
import os
import tensorflow as tf
from six.moves import urllib
def download_checkpoint(model_name: str,checkpoint_name: str,target_dir: str):
              tf.io.gfile.makedirs(target_dir)
              
              checkpoint_target = os.path.join(target_dir, checkpoint_name)
              if not os.path.exists(checkpoint_target):
                response = urllib.request.urlopen(
                  f"https://storage.googleapis.com/magentadata/models/"
                  f"{model_name}/checkpoints/{checkpoint_name}")
                data = response.read()
                local_file = open(checkpoint_target, 'wb')
                local_file.write(data)
                local_file.close()

In [ ]:
from magenta.models.music_vae import TrainedModel, configs

def get_model(name: str):

  checkpoint = name + ".tar"
  download_checkpoint("music_vae", checkpoint, "checkpoints")
  return TrainedModel(
    configs.CONFIG_MAP[name.split(".")[0] if "." in name else name],
    batch_size=8,
    checkpoint_dir_or_path=os.path.join("checkpoints", checkpoint))


/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [ ]:
!pip install visual_midi

In [ ]:
import os
import time
from typing import Union, List, Optional

import magenta.music as mm
from note_seq.protobuf.music_pb2 import NoteSequence
from visual_midi import Plotter


def save_midi(sequences: Union[NoteSequence, List[NoteSequence]],
              output_dir: Optional[str] = None,
              prefix: str = "sequence"):

  output_dir = os.path.join("output", output_dir) if output_dir else "output"
  os.makedirs(output_dir, exist_ok=True)
  if not isinstance(sequences, list):
    sequences = [sequences]
  for (index, sequence) in enumerate(sequences):
    date_and_time = time.strftime("%Y-%m-%d_%H%M%S")
    filename = f"{prefix}_{index:02}_{date_and_time}.mid"
    path = os.path.join(output_dir, filename)
    mm.midi_io.note_sequence_to_midi_file(sequence, path)
    print(f"Generated midi file: {os.path.abspath(path)}")


def save_plot(sequences: Union[NoteSequence, List[NoteSequence]],
              output_dir: Optional[str] = None,
              prefix: str = "sequence",
              **kwargs):

  output_dir = os.path.join("output", output_dir) if output_dir else "output"
  os.makedirs(output_dir, exist_ok=True)
  if not isinstance(sequences, list):
    sequences = [sequences]
  for (index, sequence) in enumerate(sequences):
    date_and_time = time.strftime("%Y-%m-%d_%H%M%S")
    filename = f"{prefix}_{index:02}_{date_and_time}.html"
    path = os.path.join(output_dir, filename)
    midi = mm.midi_io.note_sequence_to_pretty_midi(sequence)
    plotter = Plotter(**kwargs)
    plotter.save(midi, path)
    print(f"Generated plot file: {os.path.abspath(path)}")

In [ ]:
from typing import List
from note_seq.protobuf.music_pb2 import NoteSequence
from visual_midi import Coloring

def sample(model_name: str,
           num_steps_per_sample: int) -> List[NoteSequence]:

  model = get_model(model_name)
  sample_sequences = model.sample(n=2, length=num_steps_per_sample)
  save_midi(sample_sequences, "sample", model_name)
  save_plot(sample_sequences, "sample", model_name,
            plot_max_length_bar=16, coloring=Coloring.INSTRUMENT)

  return sample_sequences

In [ ]:
from magenta.music import DEFAULT_STEPS_PER_BAR
num_bar_per_sample = 16
num_steps_per_sample = num_bar_per_sample * DEFAULT_STEPS_PER_BAR 
generated_sample_sequences = sample("hierdec-trio_16bar", num_steps_per_sample)

INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, HierarchicalLstmDecoder, and hparams:
{'max_seq_len': 256, 'z_size': 512, 'free_bits': 256, 'max_beta': 0.2, 'beta_rate': 0.0, 'batch_size': 8, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [1024, 1024], 'enc_rnn_size': [2048, 2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'constant', 'sampling_rate': 0.0, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048, 2048]

INFO:tensorflow:
Hierarchical Decoder:
  input length: 256
  level output lengths: [16, 16]

INFO:tensorflow:
Decoder Cells:
  units: [1024, 1024]

INFO:tensorflow:
Decoder Cells:
  units: [1024, 1024]

INFO:tensorflow:
Decoder Cells:
  units: [1024, 1024]



/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/lstm_utils.py:99: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  name=name),
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/core.py:261: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/usr/local/lib/python3.7/dist-packages/magenta/contrib/rnn.py:751: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._names["W"], [input_size + self._num_units, self._num_units * 4])
/usr/local/lib/python3.7/dist-packages/magenta/contrib/rnn.py:754: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  initializer=tf.constant_initializer(0.0))


Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Do not call `graph_parents`.


/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/base_model.py:199: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  kernel_initializer=tf.random_normal_initializer(stddev=0.001))
/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/base_model.py:205: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  kernel_initializer=tf.random_normal_initializer(stddev=0.001))


INFO:tensorflow:Unbundling checkpoint.
INFO:tensorflow:Restoring parameters from /tmp/tmp04ud75tb/hierdec-trio_16bar.ckpt
Generated midi file: /content/output/sample/hierdec-trio_16bar_00_2022-05-02_164901.mid
Generated midi file: /content/output/sample/hierdec-trio_16bar_01_2022-05-02_164901.mid


Generated plot file: /content/output/sample/hierdec-trio_16bar_00_2022-05-02_164901.html


Generated plot file: /content/output/sample/hierdec-trio_16bar_01_2022-05-02_164901.html
